# Гревцов М.А. РТ5-61Б
### Вариант 8
### РК2 ТМО

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint as sp_randint


### Загрузка данных

In [2]:
df = pd.read_csv('googleplaystore.csv')

### Удаление пустых значений

In [3]:
df.dropna(inplace = True)

### Преобразуем данные к нужным форматам

In [4]:
df['Installs'] = df['Installs'].str.replace('+', '').str.replace(',', '').astype(int)
df['Price'] = df['Price'].str.replace('$', '').astype(float)
df['Reviews'] = df['Reviews'].astype(int)

In [5]:
df.drop('Size', axis=1, inplace=True)

### Кодирование категориальных признаков

In [6]:
label_encoder = LabelEncoder()
categorical_columns = ['Category', 'Type', 'Content Rating', 'Genres', "Last Updated", "Current Ver", "Android Ver"]

In [7]:
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

In [8]:
X = df.drop(['App', 'Rating'], axis=1)
y = df['Rating']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9360 entries, 0 to 10840
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9360 non-null   object 
 1   Category        9360 non-null   int32  
 2   Rating          9360 non-null   float64
 3   Reviews         9360 non-null   int32  
 4   Installs        9360 non-null   int32  
 5   Type            9360 non-null   int32  
 6   Price           9360 non-null   float64
 7   Content Rating  9360 non-null   int32  
 8   Genres          9360 non-null   int32  
 9   Last Updated    9360 non-null   int32  
 10  Current Ver     9360 non-null   int32  
 11  Android Ver     9360 non-null   int32  
dtypes: float64(2), int32(9), object(1)
memory usage: 621.6+ KB


### Целевая переменная и признаки


In [10]:
X = df.drop(['App', 'Rating'], axis=1)
y = df['Rating']


In [11]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Создание и применение MinMaxScaler
scaler = MinMaxScaler()

# Масштабирование данных
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Подбор гиперпараметров для DecisionTreeRegressor

In [12]:
param_dist_dt = {
    'max_depth': sp_randint(3, 10),
    'min_samples_split': sp_randint(2, 10),
    'min_samples_leaf': sp_randint(1, 10)
}

random_search_dt = RandomizedSearchCV(DecisionTreeRegressor(random_state=0), param_dist_dt, n_iter=20, cv=5, scoring='r2', random_state=0)
random_search_dt.fit(X_train_scaled, y_train)
best_dt = random_search_dt.best_estimator_

## Подбор гиперпараметров для GradientBoostingRegressor

In [13]:
param_dist_gbr = {
    'n_estimators': sp_randint(50, 200),
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': sp_randint(3, 6)
}

random_search_gbr = RandomizedSearchCV(GradientBoostingRegressor(random_state=0), param_dist_gbr, n_iter=20, cv=5, scoring='r2', random_state=0)
random_search_gbr.fit(X_train_scaled, y_train)
best_gbr = random_search_gbr.best_estimator_


### Модели

In [14]:
# Обучение моделей
best_dt.fit(X_train_scaled, y_train)
best_gbr.fit(X_train_scaled, y_train)

# Предсказания моделей
y_pred_dt = best_dt.predict(X_test_scaled)
y_pred_gbr = best_gbr.predict(X_test_scaled)

Результаты

In [15]:
# Оценка качества моделей
dt_r2 = r2_score(y_test, y_pred_dt)
gbr_r2 = r2_score(y_test, y_pred_gbr)
dt_mse = mean_squared_error(y_test, y_pred_dt)
gbr_mse = mean_squared_error(y_test, y_pred_gbr)
dt_mae = mean_absolute_error(y_test, y_pred_dt)
gbr_mae = mean_absolute_error(y_test, y_pred_gbr)
dt_rmse = np.sqrt(dt_mse)
gbr_rmse = np.sqrt(gbr_mse)

In [16]:
# Вывод результатов
print("Результаты моделей:\n")
print(f"{'Метрика':<40}{'Дерево решений':<30}{'Градиентный бустинг':<30}")
print("-" * 100)
print(f"{'R^2':<40}{dt_r2:<30.2f}{gbr_r2:<30.2f}")
print(f"{'Среднеквадратичная ошибка (MSE)':<40}{dt_mse:<30.2f}{gbr_mse:<30.2f}")
print(f"{'Средняя абсолютная ошибка (MAE)':<40}{dt_mae:<30.2f}{gbr_mae:<30.2f}")
print(f"{'Корень из среднеквадратичной ошибки (RMSE)':<40}{dt_rmse:<30.2f}{gbr_rmse:<30.2f}")

Результаты моделей:

Метрика                                 Дерево решений                Градиентный бустинг           
----------------------------------------------------------------------------------------------------
R^2                                     0.08                          0.15                          
Среднеквадратичная ошибка (MSE)         0.25                          0.23                          
Средняя абсолютная ошибка (MAE)         0.34                          0.32                          
Корень из среднеквадратичной ошибки (RMSE)0.50                          0.48                          


# Вывод

### По результатам проверки видно, что модель градиентного бустинга обходит дерево решений по всем метрикам:

1. По метрике R^2 градиентный бустинг показывает более высокое значение (0.15) по сравнению с деревом решений (0.08), что указывает на лучшее объяснение дисперсии целевой переменной.
2. По метрике среднеквадратичной ошибки (MSE) градиентный бустинг также лучше (0.23 против 0.25), что говорит о меньшей средней квадратичной ошибке предсказаний.
3. Средняя абсолютная ошибка (MAE) у градиентного бустинга ниже (0.32) по сравнению с деревом решений (0.34), что означает более точные предсказания в среднем.
4. Корень из среднеквадратичной ошибки (RMSE) также меньше у градиентного бустинга (0.48 против 0.50), что указывает на меньшие отклонения предсказаний от фактических значений.

Таким образом, модель градиентного бустинга является более точной и эффективной для данного набора данных по сравнению с деревом решений.
